In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes, svm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
fake_data = pd.read_csv('data/Fake.csv')
fake_data['label'] = 0
fake_data.info()

In [ ]:
true_data = pd.read_csv('data/True.csv')
true_data['label'] = 1
true_data.info()

In [ ]:
fake_data.isnull().sum()

In [ ]:
true_data.isnull().sum()

In [ ]:
fake_data['subject'].value_counts()

In [ ]:
true_data['subject'].value_counts()

In [ ]:
merged_data = pd.concat((fake_data, true_data))

In [ ]:
merged_data = merged_data.drop('date', axis=1)

In [ ]:
merged_data['text'] = merged_data['subject'] + ' ' + merged_data['title'] + ' ' + merged_data['text']
merged_data = merged_data.drop('subject', axis=1).drop('title', axis=1)

In [ ]:
random_permutation = np.random.permutation(len(merged_data))
merged_data = merged_data.iloc[random_permutation]

In [ ]:
merged_data = merged_data.head(1000)

In [ ]:
merged_data.info()

In [ ]:
merged_data['label'].value_counts()

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
def preprocess_text(text):
    ret_text = text.translate(text.maketrans({x:'' for x in string.punctuation})).lower() # lowercase; remove punctuation
    ret_text = nltk.tokenize.word_tokenize(ret_text)
    ret_text = [word for word in ret_text if word not in stopwords]
    ret_text = [lemmatizer.lemmatize(word) for word in ret_text]
    return  ret_text
    

In [ ]:
merged_data['text'] = merged_data['text'].apply(preprocess_text)

In [ ]:
w2v_text = Word2Vec(merged_data['text'], vector_size=100, workers=8) # TODO: test different values of vector_size, up to 300

In [ ]:
def vectorize(words, model, first_n_tokens):
    words_vecs = [model.wv[word] for word in words if word in model.wv]
    for i in range(first_n_tokens-len(words_vecs)):
        words_vecs.append(np.zeros(100,))
    return np.asarray(words_vecs[:first_n_tokens]).flatten()

In [ ]:
num_of_significant_tokens = 15 # TODO: test different values, check if 'text' is even worth it

each row now has num_of_significant_tokens * vector_size elements (15 * 100 = 1500)

In [ ]:
merged_data['text'] = merged_data['text'].apply(lambda x: vectorize(x, w2v_text, num_of_significant_tokens))

In [ ]:
merged_data['text'].head()

In [ ]:
merged_data.head()

In [ ]:
transformed_array_X = np.asarray([x for x in merged_data['text']])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(transformed_array_X, merged_data['label'], test_size=0.2, random_state=2024)

In [ ]:
print(X_train.shape)

=============================================

In [ ]:
model_bayes = naive_bayes.GaussianNB()

In [ ]:
model_bayes.fit(X_train, Y_train)

In [ ]:
y_pred = model_bayes.predict(X_test)

In [ ]:
print(accuracy_score(y_pred, Y_test))
print(precision_score(Y_test, y_pred,))
print(recall_score(Y_test, y_pred,))
print(f1_score(Y_test, y_pred,))
print(confusion_matrix(Y_test, y_pred,))

=============================================

In [ ]:
model_svm = svm.SVC(kernel='linear')

In [ ]:
model_svm.fit(X_train, Y_train)

In [ ]:
y_pred = model_svm.predict(X_test)

In [ ]:
print(accuracy_score(y_pred, Y_test))
print (precision_score(Y_test, y_pred,))
print (recall_score(Y_test, y_pred,))
print(f1_score(Y_test, y_pred,))
print(confusion_matrix(Y_test, y_pred,))

=============================================

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input

In [ ]:
model = Sequential()
model.add(Input((1,1500)))
model.add(LSTM(1500))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
X_train2 = []
for x in X_train:
    X_train2.append([x])
X_train2 = np.asarray(X_train2)

In [ ]:
model.fit(X_train2, Y_train, batch_size=100, epochs=10, validation_split=0.2)

In [ ]:
X_test2 = []
for x in X_test:
    X_test2.append([x])
X_test2 = np.asarray(X_test2)

In [ ]:
accr = model.evaluate(X_test2,Y_test)